modules nécessaire pandas scikit-learn

In [1]:
import pandas as pd
from scipy.ndimage import label
from sklearn.model_selection import train_test_split

# Chargement des données

In [2]:
df = pd.read_csv('data/faq_data.csv')
X = df['question']
y = df['answer']

# Nettoyage des colonnes 'question' et 'answer' en minuscules et suppression des ponctuations

In [3]:
import string
import re
def remove_punctuation(doc: str):
    punc = string.punctuation
    punc += '\n\r\t'
    return re.sub(' +', ' ', doc.translate(str.maketrans(punc, ' ' * len(punc))))
    

In [4]:
df = df.applymap(str.lower).applymap(remove_punctuation)
df

,question,answer
0,what is mlops,mlops is a set of practices to streamline the ...
1,why is mlops important,mlops helps in scaling monitoring and maintain...
2,what is a data drift,data drift is a change in the statistical prop...
3,how does versioning work in mlops,versioning allows you to track and manage diff...
4,what is a model registry,a model registry is a centralized storage for ...
5,how can i monitor a model in production,by tracking metrics like accuracy latency and ...
6,what is the purpose of docker,docker helps in creating reproducible environm...
7,what are ci cd pipelines,ci cd pipelines automate the testing and deplo...
8,why use mlflow,mlflow tracks experiments manages models and f...
9,how does fastapi work with ml models,fastapi allows you to build an api for serving...


# Séparation en ensembles d'entraînement et de test

In [5]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Sauvegarde des versions de données
data/train_data.csv
data/test_data.csv

In [6]:
df_train.to_csv('data/train_data.csv')
df_test.to_csv('data/test_data.csv')

### Étape 2 : Entraînement du modèle 

Entraînez un modèle de langage de type LLM sur les données d’entraînement. Suivez les expérimentations avec MLflow pour enregistrer les hyperparamètres et les performances du modèle.

### Tâche 1 : Entraînement du modèle

Utilisez un modèle préentraîné (par exemple, GPT-2 de Hugging Face).
Entraînez le modèle sur l'ensemble d'entraînement, en suivant les hyperparamètres et les métriques de performance.
Tâche 2 : Suivi des expérimentations avec MLflow

Créez une nouvelle expérience dans MLflow et enregistrez les paramètres, métriques, et le modèle.

modules nécessaire mlflow transformers[torch] datasets

# Activation mlflow server avec backend-store-uri sqlite default-artifact-root file:/home/$USER/mlops/2024/mlruns ip et port à définir

In [7]:
!pip install mlflow
!pip install datasets

In [8]:
import mlflow
import mlflow.pytorch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import os

# Désactiver wandb

In [9]:
os.environ['WANDB_DISABLED'] = 'true'

In [108]:
mlflow.set_tracking_uri('/mlruns')
mlflow_id = mlflow.create_experiment('Chatbot')
mlflow.set_experiment('Chatbot')

<Experiment: artifact_location='file:///D:/mlruns/733077457359228028', creation_time=1730979174079, experiment_id='733077457359228028', last_update_time=1730979174079, lifecycle_stage='active', name='Chatbot', tags={}>

# Charger les données d'entraînement et créer une colonne 'text'

In [27]:
df['text'] = [' '] * len(df)
df

,question,answer,text
0,what is mlops,mlops is a set of practices to streamline the machine learning lifecycle,
1,why is mlops important,mlops helps in scaling monitoring and maintaining ml models in production,
2,what is a data drift,data drift is a change in the statistical properties of the input data over time,
3,how does versioning work in mlops,versioning allows you to track and manage different versions of models and datasets,
4,what is a model registry,a model registry is a centralized storage for keeping track of model versions,
5,how can i monitor a model in production,by tracking metrics like accuracy latency and detecting data drift in real time,
6,what is the purpose of docker,docker helps in creating reproducible environments for model deployment,
7,what are ci cd pipelines,ci cd pipelines automate the testing and deployment of code and models,
8,why use mlflow,mlflow tracks experiments manages models and facilitates reproducible ml workflows,
9,how does fastapi work with ml models,fastapi allows you to build an api for serving ml models to interact with applications,


In [85]:
text_train = pd.read_csv('data/train_data.csv')
text_train['text'] = text_train['question'] + text_train['answer']
text_train = text_train.drop(columns=['question', 'answer', 'Unnamed: 0'])

# Diviser les données en ensembles d'entraînement et de validation

In [86]:
text_train, text_val = train_test_split(text_train, train_size=0.8, random_state=42)
# X_train, y_train = text_train[:, 0], text_train[:, 1]
# X_val, y_val = text_val[:, 0], text_val[:, 1]

In [87]:
text_train

,text
0,how can i monitor a model in production by tracking metrics like accuracy latency and detecting data drift in real time
7,what is the purpose of docker docker helps in creating reproducible environments for model deployment
2,what are ci cd pipelines ci cd pipelines automate the testing and deployment of code and models
4,how does fastapi work with ml models fastapi allows you to build an api for serving ml models to interact with applications
3,what is a data drift data drift is a change in the statistical properties of the input data over time
6,how does versioning work in mlops versioning allows you to track and manage different versions of models and datasets


# Convertir en Dataset et supprimer les colonnes inutiles

In [88]:
train_dataset = Dataset.from_pandas(text_train)

val_dataset = Dataset.from_pandas(text_val)

# Charger le modèle et le tokenizer, et définir un token de remplissage

In [41]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

D:\anaconda3\envs\master\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eeko\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Tokeniser le dataset avec les labels

In [89]:
tokenizer.pad_token = '[PAD]'

In [109]:
with mlflow.start_run():
    tokenized = tokenizer(train_dataset['text'], padding=True, return_tensors='pt')
    output = model(**tokenized, labels=tokenized['input_ids'])

In [110]:
!mlflow ui

^C


In [104]:
output.logits.shape

torch.Size([6, 25, 50257])

# Définir les arguments d'entraînement

In [105]:
from transformers import Trainer, TrainingArguments


# Configurer MLflow pour pointer vers votre serveur local

# Initialiser le Trainer

# Entraîner le modèle et enregistrer avec MLflow

  # Log des paramètres et du modèle dans MLflow